# AI Computer Assignment 2 (Genetic Algorithms)
Mohammad Saadati - 
_810198410_

## Introduction
Goal of this project is finding the key which a text was encoded with and decrypting the encoded text with it. To do so **Genetic Algorithm** is used. The encrypted text is given to the code along with a dicitonary of words that the encoded words after decription will be amongst them and the algorithm will solve the problem and give back the encryption key and the decoded text.

### Import Libraries
In this part, some of the necessary libraries were imported in order to use their helpful functions.

In [1]:
import numpy
import random
import re
from math import floor
import string
import time

### Defining Constants
In this part, constant values are defined in order to make the code more readable and more flexible to change.

In [2]:
POPULATION_SIZE = 100  
NUMBER_OF_GENERATIONS = 1500 
MUTATION_PROBABILITY = 0.08
CROSSOVER_PROBABILITY = 0.3
MAX_STALE_COUNT = 100
CONSTANT = 14
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
            'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

## Phase 0: Cleaning Data & Create Dictionary
In this part, we first convert all the uppercase characters of text to lowercase in order to simplifying algorithm and then replace all things except alphabet with space, after that we split data with space, and finally, we convert a list of words to map to delete duplicated words. Now we have a dictionary which we can used it to find the fitness of each Chromosome.

## Phase 1: Definition of Gene & Chromosome
**Gene**: each letter in key (chromosome) represents as a gene.

**Chromosome**: a sequence of letter (genes) with default size of *14* represent as a chromosome.

In [3]:
class Chromosome(object):
    def __init__(self, _chromosome):
        self.fitness = 0
        self.chromosome = _chromosome
    
    def update_fitness(self, fitness):
        self.fitness = fitness
            
    def mutation(self, mutation_rate, intial_input):
        None       

## Phase 2: Initializing Population
Initialized population of size *100* randomly using the `initiate_population`. We shuffle the population in order to increase diversity; this is done by using the `shuffle` function from Random library in python.

## Phase 3: Implementing Fitness Function
Fitness defiend in this problem is the accuracy of our prediction for each chromosome. The fitness of a chromosome solution is determined by how close it is to being the actual solution for the problem. For finding the fitness value of each chromosome, we defined `calculate_fitness` function. In this function, we decode the encoded text and then calculate how many words in the decoded text is in the dictionary. The number of these words is the fitness value for each chromosome. Finally we sort our population based on chromosome fitness in reverse mode because we want to choose the best of them for the next generation.

Fitness function should be sufficiently fast to compute and must quantitatively measure how fit a given solution is or how fit individuals can be produced from the given solution. Our fitness function satisfies these criteria and its suitable for this problem.

## Phase 4: Implementing Crossover & Mutation Function and Generate the Next Population
### Crossover
Crossover relates to the analogy of genes within each parent chromosome mixing together in the hopes of creating a fitter child chromosome. Create two new child chromosomes by crossing over parent genes. This makes it possible that if part of the selected genes are suitable and part is inappropriate, the improper part is likely to be discarded, thus combining the part that we think is appropriate with another part of the chromosome that brings us closer to the answer. We consider 0.3 for the crossover probability and it is constant during the execution of the algorithm.

The crossover function requires two parents to be selected from the population pool. Pick two chromosomes from the population based on their rank and get them to compete against each other. In the `crossover` function, we choose one points and then copy the values between this points to the offspring and then complete the chromosome. The crossover happens in *70%* of the population.
### Mutation
The mutation function is applied to a chromosome, using a method to mutate and modify it in the hope that it can achieve a better chromosome. This operation is performed after each crossover, because we may be stuck in a certain state every time after this operation. By slightly changing these chromosomes, we can get out of this state and get closer to the answer. we have chosen a variable *mutation_rate* which is the mutation probabality and its default value is *0.08* to make more change to chromosomes. Whenever we are close to the solution, we decrease it by *0.002* and whenever we are in a local maximum, we increase it by *0.002*.

In the `mutation` function, we mutate a chromosome by picking a gene, and then replace it with a new random gene. The initial mutation rate is *0.08*.

### Next Population
With all the above functions implemented we can generate the new population. First we need all of the fitness values in sorted order so we could know and rank chromosomes. After passing some of the chromosomes based on elitism probability, the remaining chromosomes will be chosen to be crossed over or mutated. We use rank base selection for this project, because of the *FPS* problem in making chromosome bios. Also, We use **ELITISM** selection method. This means we copy *30%* of the best chromosomes to the next generation, the remain, *70%*, used for crossover and mutation.

## Phase 5:
Load a file to decode. 

For each generation, calculated the fitness of each candidate and sorted them. Then traverse all the chromosomes to check for solution. if solution not found, select the fittest chromosomes and preserve them for the next generation ( *30* elites in new generation). Took first *30* as elites(elites would definately be part of the next generation). For the remaining *70*, randomly selected two candidates from the population and performed crossover among them. Mutation is done on each candidate. Then append elites into the end of the population. These will not have been affected by crossover or mutation. Calculate new adaptive mutation rate. We continue doing this until we get a fitness value of *1*. This indicates we have successfully found a solution to the decoding problem. To avoid biasing chromosomes, we can Re-generate the population if for example *100* generations have passed with the fittest two chromosomes always having the same fitness.

In [4]:
class Decoder(object):
    def __init__(self, global_text, encoded_text, _key_lenght = CONSTANT):
        self.global_text_dict = self.clean_text(self.lower_text(global_text))
        self.encoded_text = encoded_text
        self.encoded_text_word = self.split_text(self.lower_text(encoded_text))
        self.encoded_text_word = [w for w in self.encoded_text_word if w != '']
        self.key_lenght = _key_lenght
        self.population = []
        self.number_of_elites = floor(CROSSOVER_PROBABILITY * POPULATION_SIZE)
        self.best_fitness = -1
        self.best_chromosome = None
        self.mutation_rate = MUTATION_PROBABILITY
        self.phi = 0
        self.number_of_mutations = 0
        
    def lower_text(self, text):
        return (text).lower()
    
    def split_text(self, text):
        return re.split('[^a-zA-Z]', text)
    
    def make_dict(self, words):
        return set(words)
    
    def clean_text(self, text):
        words = self.split_text(text)
        return self.make_dict(words)
    
    def initiate_population(self):
        print("generating population...\n")
        population = []
        for i in range(POPULATION_SIZE):
            key = random.sample(ALPHABET, self.key_lenght)
            while key in population:
                random.shuffle(key)
            self.population.append(Chromosome(''.join([str(elem) for elem in key])))
        for i in range(POPULATION_SIZE):
            self.population[i].update_fitness(self.calculate_fitness(self.population[i].chromosome))
        print("population generated.\n")
            
    def get_fitness(self, chromosome: Chromosome):
        return chromosome.fitness
    
    def calculate_fitness(self, key):
        counter = 0
        decoded_words = []
        for word in self.encoded_text_word:
            decoded_word = ""
            for letter in word:
                cur_key = ord(key[counter]) - 97
                letter = ord(letter) - 97
                decoded_word += chr(((letter - cur_key) % 26) + 97)
                counter = (counter + 1) % self.key_lenght
            decoded_words.append(decoded_word)
        is_in_dict = 0
        for w in decoded_words:
            if w in self.global_text_dict:
                is_in_dict += 1
        return is_in_dict / len(decoded_words)
    
    def evaluate_fitnesses(self):
        for i in range(POPULATION_SIZE):
            self.population[i].update_fitness(self.calculate_fitness(self.population[i].chromosome))   
            
    def crossover(self, first_chromosome, second_chromosome):
        p = floor(self.key_lenght / 2)
        new_key1 = first_chromosome.chromosome[:p] + second_chromosome.chromosome[p:]
        new_key2 = first_chromosome.chromosome[p:] + second_chromosome.chromosome[:p]
        result1 = Chromosome(new_key1)
        result1.update_fitness(self.calculate_fitness(new_key1))
        result2 = Chromosome(new_key2)
        result2.update_fitness(self.calculate_fitness(new_key2))
        return result1, result2
    
    def mutation(self, chromosome):
        if random.random() < self.mutation_rate:
            self.number_of_mutations += 1
            p = random.randint(0, self.key_lenght - 1)
            chromosome_key = chromosome.chromosome
            new_chromosome_key = chromosome_key[:p] + random.choice(string.ascii_lowercase) + chromosome_key[p+1:]
            new_chromosome_fitness = self.calculate_fitness(new_chromosome_key)
            if new_chromosome_fitness > chromosome.fitness:
                self.phi += 1
                result = Chromosome(new_chromosome_key)
                result.update_fitness(new_chromosome_fitness)
                return result
            else:
                return chromosome
        else:
            return chromosome
    
    def decode_text(self, key):
        counter = 0
        decoded_text = ""
        for i in range(len(self.encoded_text)):
            if ord(self.encoded_text[i]) >= 97 and ord(self.encoded_text[i]) <= 122:
                cur_key = ord(key[counter]) - 97
                encoded_letter = ord(self.encoded_text[i]) - 97
                decoded_text += chr(((encoded_letter - cur_key) % 26) + 97)
                counter = (counter + 1) % self.key_lenght
            elif ord(self.encoded_text[i]) >= 65 and ord(self.encoded_text[i]) <= 90:
                cur_key = ord(key[counter]) - 97
                encoded_letter = ord(self.encoded_text[i]) - 65
                decoded_text += chr(((encoded_letter - cur_key) % 26) + 65)
                counter = (counter + 1) % self.key_lenght
            else:
                decoded_text += self.encoded_text[i]
        return decoded_text
         
    def decode(self):
        start = time.time()
        sigma = 1
        stale = 0
        self.phi = 0
        self.number_of_mutations = 0
        self.initiate_population()
        for generation in range(NUMBER_OF_GENERATIONS):
            print("------------------------------------------------")
            self.population = sorted(self.population, key=self.get_fitness, reverse=True)
            self.best_fitness = self.population[0].fitness
            self.best_chromosome = self.population[0].chromosome
            print("Generation %d" % generation)
            if self.best_fitness == 1:
                print("Solution found!")
                print("\nKey:", self.best_chromosome)
                print("\nThe time duration was:",int((time.time() - start)/60), "min and", '%.2f'%((time.time() - start)%60), " secs")
                break
            print("Key:", self.best_chromosome, " Fitness: %f" % self.best_fitness)
            for_crossover = [chromosome for chromosome in self.population[:(POPULATION_SIZE - self.number_of_elites)]]
            random.shuffle(for_crossover)
            next_population = [self.mutation(chromosome) for chromosome in self.population[:self.number_of_elites]]
            while len(next_population) < POPULATION_SIZE:
                chromosome_parent1 = for_crossover.pop(0)
                chromosome_parent2 = for_crossover.pop(0)
                chromosome_child1, chromosome_child2 = self.crossover(chromosome_parent1, chromosome_parent2)
                next_population.append(self.mutation(chromosome_child1))
                next_population.append(self.mutation(chromosome_child2))  
            self.population = next_population 
            if(self.number_of_mutations != 0):
                self.phi = self.phi / self.number_of_mutations  
            else:
                self.phi = 0       
            if(self.phi < 0.2):
                sigma = sigma / 0.998   
            if(self.phi > 0.2):
                sigma = sigma * 0.998  
            self.mutation_rate = abs(numpy.random.normal(loc = 0.0, scale = sigma, size = None))
            while 1 < self.mutation_rate:
                self.mutation_rate = abs(numpy.random.normal(loc = 0.0, scale = sigma, size = None))   
            self.phi = 0
            self.number_of_mutations = 0
            if(self.population[0].fitness != self.population[1].fitness):
                stale = 0
            else:
                stale += 1
            if(stale >= MAX_STALE_COUNT):
                print("The population has gone stale. Re-seeding...")
                self.initiate_population()
                self.phi = 0
                self.number_of_mutations = 0
                self.mutation_rate = MUTATION_PROBABILITY
                stale = 0
                sigma = 1    
        return self.decode_text(self.best_chromosome)

encodedText = open("encoded_text.txt").read()
globalText = open("global_text.txt").read()
d = Decoder(globalText, encodedText)
decodedText = d.decode()

print("\nThe decoded text is:\n")
print(decodedText)

generating population...

population generated.

------------------------------------------------
Generation 0
Key: nzikehacjmxfyl  Fitness: 0.062382
------------------------------------------------
Generation 1
Key: nzikehacjmxfyl  Fitness: 0.062382
------------------------------------------------
Generation 2
Key: nzikehacjmxfyl  Fitness: 0.062382
------------------------------------------------
Generation 3
Key: bgiftucftsbsao  Fitness: 0.066163
------------------------------------------------
Generation 4
Key: zbofyidagntevi  Fitness: 0.075614
------------------------------------------------
Generation 5
Key: zbofyidagntevi  Fitness: 0.075614
------------------------------------------------
Generation 6
Key: zbofyidagntevi  Fitness: 0.075614
------------------------------------------------
Generation 7
Key: zbofyidagntevi  Fitness: 0.075614
------------------------------------------------
Generation 8
Key: zbofyidmdrteyd  Fitness: 0.081285
------------------------------------------

------------------------------------------------
Generation 80
Key: atberteinstein  Fitness: 0.661626
------------------------------------------------
Generation 81
Key: azberteinstein  Fitness: 0.663516
------------------------------------------------
Generation 82
Key: azberteinstein  Fitness: 0.663516
------------------------------------------------
Generation 83
Solution found!

Key: alberteinstein

The time duration was: 0 min and 48.47  secs

The decoded text is:

Albert Einstein
Old Grove Rd.
Nassau Point
Peconic, Long Island

August 2nd, 1939

F.D. Roosevelt,
President of the United States,
White House
Washington, D.C.

Sir:

Some recent work by E. Fermi and L. Szilard, which has been communicated to me in manuscript, leads me to expect that the element uranium may be turned into a new and important source of energy in the immediate future. Certain aspects of the situation which has arisen seem to call for watchfulness and, if necessary, quick action on the part of the Administ

## Phase 6: Questions 
### 1.
For arbitrarily large population sizes the accuracy of the genetic algorithm approaches, but does not reach, *100%*. The greater the population size the greater the chance that the initial state of the population will contain a chromosome representing the optimal solution.

For small population sizes, mutation will have to be more prominent in order to compensate for smaller population
### 2.
The increase in population size causes the number of generations to converge to increase. This appears to be due to the overall increase in probability that mutation will occur. If mutation occurs for large population sizes, more generations are needed to eliminate the mutated chromosomes. Increase of the population size, above *100* chromosomes, does not improve the solution accuracy. Moreover, the computational time is increased significantly. If we increase the population size at each iteration, both speed of convergence and accuracy will increase. The variable population-size genetic algorithm is more efficient than the standard genetic algorithm.
 
### 3.
The most important effect of mutation is to generate new chromosomes using random abilities, increase the diversity in
the population and to prevent getting stuck in local maximums or at least lower probability of it. So without mutation,the individuals in the population will be too similar, the speed of the algorithm will be reduced and in some cases we simply don't reach the goal because there would be a kind of bias in chromosomes after tens or hundreds iterations. 

If we use just crossover in the genetic algorithm, its very potential to stuck in the local extremum for a long time. So, the mutation is an important part of the genetic algorithm.

### 4.
Crossover is more effective than mutation and causes more accuracy. Crossover leads to making a new generation. It makes completely new solutions and leads to significant improvement in the fitness scores. But, as we said, using just crossover may lead to stuck in some situations. So, mutation helps to change the current answer a little to make a possibly better generation. As we saw experimentally and as it's resonable, mutation is more effective in small population-size because of the ability of generating new chromosomes even when chromosomes are similar.
### 5.
If this happens to us, it means that we are stuck at a local maximum point and we have to get out of this point. To prevent chromosomes stability, when maximum fitness of our model gets below 90% we increase mutation and crossover probability to cause more changes and if maximum fitness of our model remains the same for a number of generation we increase mutation and crossover probability to cause more changes. Also we can store history of generations and if we are repeating on a fitness for example for 100 times Then i change the probabilty of mutationRate for one generation to get out of this situation. 
### 6.

### 7.
Remove stop words from the dictionary to search for the word decoded faster in the dictionary.